# The Toggle switch

The genetic toggle switch `(Gardner et al. Nature 403, 339‐342 (2000))` is a synthetic gene regulatory network implemented in E. coli. it is able to show bistability between an ON and OFF states in the trascription of two genes that mutually repress each other trascription. 

The significance of bistability in this system is that that a switch can be constructed with the two steady states forming the ON and OFF states of the switch. Once prepared in a state the system remains there for long periods of time. By adding small molecules known as inducers, or by applying heat, the authors were able to alter the system such that it switched between the ON and OFF states. This system demonstrates the basis for cellular memory and has since been used to form synthetic biological devices that can 'remember' an event in the past.

Figure 1.5.1 shows the genetic regulatory network that comprises this system. The symbol $\rightarrow$ indicates activating interactions and $\dashv$ represents repressive or inhibitory interactions. The system is formed from two repressor proteins. Repressor 1 inhibits transcription of Repressor 2 by binding to promoter 1. Repressor 2, inhibits transcription of Repressor 1 by binding to promoter 2. The mutually repressing processes are what gives rise to the bistable behaviour of the system; either Repressor 1 wins and we have only these proteins present in the system or Repressor 2 wins and we have only these proteins present. This are the two observable steady states of the system.
We can also see how the inducer reactions allow switching between the states. Imagine we are in the state where Repressor 1 is present and Repressor 2 is absent. The action of adding Inducer 1 hinders the repression of Repressor 2. This allows Repressor 2 proteins to build up eventually to the levels where Repressor 1 is completely suppressed. Thus the system moves into the state where Repressor 2 is present and Repressor 1 is absent. If the system is in this state then Inducer 2 can be used to perform the reverse switch.
The final piece of the network is the reporter gene which in this case codes for a green fluorescent protein (GFP). This is downstream of Promoter 2 so that when Repressor 1 is expressed so is GFP. This fluorescence can be used to experimentally identify the state of the system within individual cells.

$$\begin{align}        
            \frac{ du }{dt} &= \frac{\alpha_1}{1 + v^{\beta} } - u  \tag{1} \\ 
            \frac{ dv }{dt} &= \frac{\alpha_2}{1 + u^{\gamma} } - v   \tag{2}  \\
\end{align}
$$


where $u,v$ represent the concentrations of the two proteins in the system (Repressor 1 and Repressor 2), $\alpha_1 , \alpha_2$ are the effective rates of synthesis of Repressor 1 and 2 respectively. $\beta, \gamma$ are the cooperativity of repression of Promoter 1 and Promoter 2 respectively, i.,e,  Repressor 1 $u$ multimerizes with γ subunits and repressor 2 $v$ forms multimeres of β monomers. The form of these equations should be familiar from Gene Expression 1 and Gene Expression 2. Essentially for each protein we have a production term and a decay termThis is an input-output model., with the production terms modelled via two Hill functions which take into account the repression by the other protein, $u \dashv v$ and $v \dashv u$.

In [20]:
using DifferentialEquations
using Plots; gr();

In [16]:
tspan = (0,10)
α_1 = 10.0;
α_2 = 9.0;
β = 2.0;
γ = 2.1;
u0=[2,0.01] # it fails when u[1] and u[3] are similar
p=[α_1,α_2,β,γ];

In [17]:
function Toggleswitch1!(du,u,p,t)
    α_1,α_2,β,γ = p
    du[1] = -u[1] + α_1/(1+(u[2]^β))
    du[2] = -u[2] + α_2/(1+(u[1]^γ))
end

Toggleswitch1! (generic function with 1 method)

In [18]:
tspan = (0.0,15)
α_1=10
α_2=10
β=2
γ=2
u0=[0.01,1] # it fails when u[1] or u[2] are zero
p=[α_1,α_2,β,γ];

In [19]:
prob1 = ODEProblem(Toggleswitch1!,u0,tspan,p)
sol1 = solve(prob1)
plot(sol1,label=["u","v"])

ErrorException: error compiling _plot!: error compiling _display: could not load library "libGR.so"
dlopen(libGR.so.dylib, 1): image not found

Here we see that the system reaches and equilibrium where $v =10$ and $u=0$, i.e., one of the genes is being transcribed and the other no. Interestingly, when we change the intial condition. 

In [8]:
u0=[2,0.9] # it fails when u[1] or u[2] are zero
prob2 = ODEProblem(Toggleswitch1!,u0,tspan,p)
sol2 = solve(prob2)
plot(sol2,label=["u","v"])

ErrorException: error compiling _plot!: error compiling _display: could not load library "libGR.so"
dlopen(libGR.so.dylib, 1): image not found

Now we see that th other variable $u =10$ while $u=0$. So, the state of activation of the system depends on the initial condition. This is a hallmark of a bistable regime: a region in the parameters space where two different solutions are stable, and the system evolves towards one or the other depending on the initial conditions.

We can do the same using the DLS notation, that is simple to read and to manipulate

In [9]:
Toggleswitch3! = @ode_def ab begin
    du = -u + α_1/(1+(v^β))
    dv = -v + α_2/(1+(u^γ))
    end α_1 α_2 β γ

(::ab{getfield(Main, Symbol("##3#7")),getfield(Main, Symbol("##4#8")),getfield(Main, Symbol("##5#9")),Nothing,Nothing,getfield(Main, Symbol("##6#10")),Expr,Expr}) (generic function with 2 methods)

In [10]:
u0=[2,0.9] # it fails when u[1] or u[2] are zero
prob3 = ODEProblem(Toggleswitch3!,u0,tspan,p)
sol3 = solve(prob3)
plot(sol3,label=["u","v"])

ErrorException: error compiling _plot!: error compiling _display: could not load library "libGR.so"
dlopen(libGR.so.dylib, 1): image not found

## Stability analysis of the Toggle switch

First,we will calculate and plot the Null-clines from Eqs 1-2 as

$$\begin{align}        
            u &= \frac{\alpha_1}{1 + v^{\beta} }  \tag{3} \\ 
            v &= \frac{\alpha_2}{1 + u^{\gamma} }  \tag{4}  \\
\end{align}
$$

Soving Eq. 4 for u, 

$$\begin{align}        
            1 + u^{\gamma} &= \frac{\alpha_2}{v }  \notag  \\
            u^{\gamma} &= \frac{\alpha_2}{v }-1  \notag  \\
            u^{\gamma} &= \frac{\alpha_2 -v}{v }  \notag  \\
            u &= (\frac{\alpha_2 -v}{v })^{1/\gamma} \tag{5}  \\
\end{align}
$$

In [11]:
u0=[10,9.999]
α_1=10
α_2=10
β=2
γ=2
v_vector = LinRange(0,u0[2],100)
u_nullcline= α_1 ./ (v_vector.^β .+1)
v_nullcline= ((α_2 .- v_vector) ./ v_vector).^(1/γ)
plot(v_vector,[u_nullcline,v_nullcline],label=["Nullcline u","Nullcline v"])

ErrorException: error compiling _plot!: error compiling _display: could not load library "libGR.so"
dlopen(libGR.so.dylib, 1): image not found

## Short Reminder of stability analysis of nonlinear systems

### Stability in linear systems

Next,  we will analize the stabilty of the system. As a reminder, any given system of linear differential equations can be written as:

$$
\frac{d\vec{x}}{dt}= A \vec{x} \tag{1}
$$

where $\vec{x}$ represents a point in the phase plane and $\frac{ d \vec{x} }{dt} $ ̇ represents the velocity vector at that point. The fixed points, $\vec{x}^∗$, for this system are the points that satisfy $A \vec{x}^∗ = 0$, meaning that all time derivatives are zero. The fixed points represent the steady state of the system.

In the case of just two dimensions we have:
$$
\binom{\dot{u}}{\dot{v}}=\begin{pmatrix}
 a&b \\ 
 c&d 
\end{pmatrix}\binom{u}{v}
$$

which is equivalent to write

$$\begin{align}        
            \frac{ du }{dt} &= a u + b v  \notag \\ 
            \frac{ dv }{dt} &= c u + d v      \\
\end{align}
$$

### Stability in nonlinear systems

On the other hand, any nonlinear system of equations can be expressed in general as :

$$
\begin{align}        
           \frac{ d \vec{x} }{dt} &= f(\vec{x})  \tag{5} \\ 
\end{align}
$$
where $\vec{x} = (x_1,x_2,...x_n)$, and $f(\vec{x}) = (f_1(\vec{x}),f_2(\vec{x})...f_n(\vec{x}))$. Again, the fixed points, $\vec{x}^∗$  are the points that satisfy $f(\vec{x}^∗) = 0$, meaning that all time derivatives are zero.

In the case of two dimensions, we write:

$$\begin{align}        
           \frac{ du }{dt} &= f(u,v)  \tag{5} \\ 
            \frac{ dv }{dt} &= g(u,v)  \tag{6}  \\
\end{align}
$$

To analyze the stability of the system, we have to be able to write the system of nonlinear equations in linear form (i.e., similar to Eq. 1). Therefore, we have to linearize our nonlinear set of equations aroudn the stady state of the system. This is done as follows:

$$
\binom{\dot{u}}{\dot{v}}=\begin{pmatrix}
 a&b \\ 
 c&d 
\end{pmatrix}\binom{u}{v}=\begin{pmatrix}
  \frac{ df }{du}& \frac{ d f }{dv} \\ 
  \frac{ dg }{du}& \frac{ dg }{dv} 
\end{pmatrix}\binom{u}{v}
$$

This matrix is called the Jacobian matrix. This matrix can be used to classify the nature of the fixed points either in linear or nonlinear systems of equations. The dynamics of the system around the fixed points is characterized by the value of the Trace $Tr(A)$ and the determinant $Det(A)$ of the Jacobian matrix.

For a linear system the point $\vec{x}^*$ = 0 will always be a fixed point. These fixed points can have different behaviours as; nodes, spirals, centers, stars, non-isolated fixed points, saddle points or degenerate nodes. To clasify a fixed point, we have to look at the trace and teh determinato of the linearize matrix $A$:
$$
\begin{align} 
Tr(A) &=a+d \\
det(A) &= ad − bc
\end{align}
$$

Let’s try to find a solution of the convenient form in the form of eigenvectors and eigenvalues. 
$$
\binom{\dot{u}}{\dot{v}}=\begin{pmatrix}
 \lambda_1&0 \\ 
 0&\lambda_2 
\end{pmatrix}\binom{u}{v}
$$

To calculate teh eigen vectors, we perform the following computation:
$$
det(A-\lambda I)=det\begin{pmatrix}
 a-\lambda&b \\ 
 c&d- \lambda
\end{pmatrix}=0
$$
the Trace and the deteriminat of the lienarized matrix takes the simple form:
$$
\begin{align} 
Tr(A) &=\lambda_1+\lambda_2 \\
det(A) &= \lambda_1 \lambda_2
\end{align}
$$

the rule for a fixed point to be stable is that $Tr A < 0$ and $Det A > 0$, which is equivalent to say that the real part of all eigenvalues Re($\lambda$) are lower than zero. In addition, if the eigenvalues are real, trajectories are direct
if the complex, the trajectory will be a spiral. 





## Stability analysis of the Toggle switch 

So, based on the theory of stability of nonlinear systems of equations, we write the Jacobiona Matrix of the system of diferential equations corresponding to the Toggle switch model as:
$$
A=\begin{pmatrix}
 -1 & -\frac{\alpha_1 \beta v^{\beta-1}}{(1 + v^{\beta})^2 } \\ 
 -\frac{\alpha_2 \gamma u^{\gamma-1}}{(1 + u^{\gamma})^2 }&-1 \end{pmatrix}
$$

The trace of the Jacobian is always negative. Therefore, the steady state solutions will be stable if the determinant of the Jacobian matrix is larger than zero $Det A > 0$. By solving the determinant we will find the conditions (i.e., the values of the parameters) that set teh stady states as stable or inestable. This would also define the boundary in parameter space that separates the bistable from monostable region. 

if we solve $Det(A)=0$, we obtain the following expression:
$$
\frac{\alpha_1 \beta v^{\beta-1}}{(1 + v^{\beta})^2 }\frac{\alpha_2 \gamma u^{\gamma-1}}{(1 + u^{\gamma})^2 }=1
$$


In [12]:
using Pkg
Pkg.add("Interact")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed Missings ──────────── v0.4.0
 Installed PoissonRandom ─────── v0.4.0
 Installed RecursiveArrayTools ─ v0.18.6
 Installed DiffEqCallbacks ───── v2.5.2
 Installed DelayDiffEq ───────── v4.7.1
 Installed ForwardDiff ───────── v0.10.2
 Installed StochasticDiffEq ──── v5.11.1
 Installed OrdinaryDiffEq ────── v4.21.0
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [bcd4f6db] ↑ DelayDiffEq v4.7.0 ⇒ v4.7.1
  [459566f4] ↑ DiffEqCallbacks v2.5.0 ⇒ v2.5.2
  [f6369f11] ↑ ForwardDiff v0.10.1 ⇒ v0.10.2
  [e1d29d7a] ↑ Missings v0.3.1 ⇒ v0.4.0
  [1dea7af3] ↑ OrdinaryDiffEq v4.19.0 ⇒ v4.21.0
  [e409e4f3] ↑ PoissonRandom v0.3.1 ⇒ v0.4.0
  [731186ca] ↑ RecursiveArrayTools v0.18.5 ⇒ v0.18.6
  [789caeaf] ↑ StochasticDiffEq v5.11.0 ⇒ v5.11.1


In [13]:
Central_dogma1! = @ode_def abetterway begin
  dM = k_M * T * D   - γ_M * M
    end k_M T D γ_M

(::abetterway{getfield(Main, Symbol("##11#15")),getfield(Main, Symbol("##12#16")),getfield(Main, Symbol("##13#17")),Nothing,Nothing,getfield(Main, Symbol("##14#18")),Expr,Expr}) (generic function with 2 methods)